<img src="logo.png">

# Diagnósticos de regresión.

Para que el modelo de regresión lineal sea fiable, en las conclusiones derivadas de las estimaciones de intervalos e inferencias (intervalos de confianza, contrastes de hipótesis,...) que realizamos a partir de dicho modelo se tienen que verificar cietas hipótesis.

Las tareas que realizan dichas verificacones se denominan **diagnósticos de regresión**.

Estos se clasifican en tres categorías:

* Errores: los errores deben seguir una $N(0,\sigma)$ con la misma varianza y ser incorrelacionados.


* Modelo: los puntos se tienen que ajustar a la estructura lineal considerada.


* Observaciones anómalas: a veces unas cuántas observaciones no se ajustan al modelo y hay que detectarlas.


En general hay dos métodos: gráficos y numéricos. 

## Normalidad de los residuos.

Para detectar la normalidad de los residuos, podemos aplicar todos los tests de normalidad vistos en el tema de Bondad de Ajuste:

* Test Kolmogorov-Smirnov-Lilliefors.


* Test Anderson-Darling.


* Test de Shapiro-Wilks.


* Test D'Agostino-Pearson.

### Normalidad de los residuos en R

En **R**, si ``r`` es el modelo, entonces ``r$residuals`` es el vector de errores. Por lo tanto es a este a vector a quien le tenemos que hacer la prueba de gaussianidad. Recuerda que los test de gaussianidad requieren la paquetería ``fBasics`` y se sigue la regla de decisión

$p$-valor|Decisión|Significado
:--|:--|:--
Pequeño|Rechazar $H_0$|Hay buena probabilidad de que **NO es gaussiana**
Grande|Rechazar $H_1$|Hay buena probabilidad de que **SÍ es gaussiana**


## Homocedasticidad.

Para verificar la homocedasticidad de los residuos, podemos aplicar el **Test de Breuch-Pagan**. Su aplicación es bastante sencilla:

* Obtener los residuos $\{e_i\}_{i=1}^n$ de la regresión lineal inicial.


* Calcular el coeficiente de determinación $R^2$ de la regresión lineal de los $e_i^2$ respecto de las variables iniciales. Es decir, calcular el coeficiente de determinación del modelo siguiente: $$E^2=\gamma_0+\gamma_1X_1+\cdots+\gamma_kX_k$$


* Calcular el estadístico $X_0=nR^2$ el cual, suponiendo homocedasticidad, sigue una $\chi^2_k$.


* Calculamos el p-valor $P(\chi^2_k\ge X_0)$ con el significado usual.


La regla de decisión es 

$p$-valor|Decisión|Significado
:--|:--|:--
Pequeño|Rechazar $H_0$|Hay buena probabilidad de que los residuos **NO sean homocedásticos**
Grande|Rechazar $H_1$|Hay buena probabilidad de que los residuos **SÍ sean homocedásticos**

También se puede hacer una prueba gráfica: cuando se grafica la nube de puntos de los valores ajustados (es decir, las $\hat{y_i}$) vs los residuos, deberemos observar un **cielo estrellado** centrado en la recta $y=0$.

### Homocedasticidad en R

Para realizarlo en R, simplemente usamos ``bptest(r)`` de la paquetería ``lmtest``, donde ``r`` es el objeto de R donde hemos guardado la información de la regresión original.

Y para realizar la gráfica hacemos

``ggplot(data = datos, aes(r$fitted.values, r$residuals)) +
    geom_point() +
    geom_hline(yintercept = 0) +
    theme_bw()``

donde ``datos`` es tu tabla original.

## Autocorrelación de los residuos: Test de Durbin-Watson.

Otra de las hipótesis que se deben verificar para que el análisis de regresión sea correcto es la incorrelación de los residuos.

La autocorrelación de los residuos puede ser de dos tipos:

* Positiva: un valor positivo (negativo) de un error genera una cadena de residuos positivos (negativos)


* Negativa: los residuos alternan signo.


Para comprobar que los residuos no presentan correlación, se puede aplicar el Test de Durbin-Watson.

Sean $\{e_i\}_{i=1}^n$ los residuos de la regresión. Tomemos $E_i$ y $E_{i-1}$ como las variables aleatorias de error y consideremos la recta de regresión de $E_i$ respecto de $E_{i-1}$. Esto es, consideremos el modelo $E_i=\beta_0+\beta_1E_{i-1}$.

Se plantea el contraste $$\left\{\begin{array}{l}\mathcal{H}_0:\beta_1=0\\\mathcal{H}_1:\beta_1\neq0\end{array}\right.$$ con el estadístico de contraste $$d=\frac{\sum_{i=2}^n(e_i-e_{i-1})^2}{\sum_{i=1}^ne_i^2}$$

El valor de este estadístico es aproximadamente $2(1-b_1)$ donde $b_1$ es una estimación de $\beta_1$. Si $\mathcal{H}_0$ es cierta, su distribución es la de una cierta combinación lineal de $\chi^2$.

La regla de decisión es

$p$-valor|Decisión|Significado
:--|:--|:--
Pequeño|Rechazar $H_0$|Hay buena probabilidad de que los residuos **SÍ tengan autocorrelación**
Grande|Rechazar $H_1$|Hay buena probabilidad de que los residuos **NO tengan autocorrelación**

### Autocorrelación de los residuos en R

Este test está implementado en la función ``dwt`` del paquete ``car``. Su sintaxis es ``dwt(r,alternative="two.sided")``.

## Aditividad y linealidad

Sea $i$ fija y consideremos y perturbemos la variable $X_i$, digamos con $x_i+h$. Entonces

$$
\begin{array}{rcl}
\boldsymbol{Y|_{x_1,...,x_i+h,...x_k}-Y|_{x_1,...,x_i,...x_k}}&\boldsymbol{=}&\boldsymbol{\beta_0+\beta_1x_1+...+\beta_i(x_i+h)+...+\beta_kx_k-(\beta_0+\beta_1x_1+...+\beta_ix_i+...+\beta_kx_k)}\\
&\boldsymbol{=}&\boldsymbol{\beta_ih}
\end{array}
$$


Cuando se plantea un modelo lineal, se suponen implícitamente las condiciones siguientes:

* Aditividad: la variación en la expresión en **negrita** es independiente de las variables que no se perturbaron.

* Linealidad: la variación en la expresión en **negrita** es la misma independientemente de qué variable se perturbó. 



## Aditividad.

Podemos comprobar la aditividad con el Test de Tukey usando los llamados gráficos de residuos parciales. La idea principal es verificar que no haya interacción entre las variables independienes y así, cada una tendrá un efecto aditivo en el modelo.

Si existe la interacción, algunos de los términos cuadráticos tendrán peso en el modelo. Esta es la base del Test de Tukey.

Para aplicarlo, se siguen los siguientes pasos:

* Se obtienen los valores ajustados $\{\widehat{y_i}\}$ por la regresión lineal inicial.


* Se lleva a cabo una segunda regresión lineal incluyendo como nueva variable independiente los $\widehat{y_i}^2$. Sea $\beta$ el coeficiente de esta nueva variable.


* Se testea si la variable $\widehat{y}^2$ es significativa en la segunda regresión. Es decir, se realiza el contraste $$\left\{\begin{array}{l}\mathcal{H}_0:\beta=0\\\mathcal{H}_1:\beta\neq0\end{array}\right..$$


Si no podemos descartar $\mathcal{H}_0$, la variable de los $\widehat{y_i}^2$ no es significativa y el modelo es aditivo.

La regla de decisión es

$p$-valor|Decisión|Significado
:--|:--|:--
Pequeño|Rechazar $H_0$|Hay buena probabilidad de que el modelo **NO es aditivo**
Grande|Rechazar $H_1$|Hay buena probabilidad de que el modelo **SÍ es aditivo**

### Aditividad en R

En R, para realizar el Test de Tukey hay que usar la función ``residualPlots`` del paquete ``car``:

``residualPlots(r,plot=...)``

donde 

``r`` es el objeto de R donde hemos guardado la información de la regresión original

``plot`` es un parámetro lógico. Si vale TRUE, nos dibuja los gráficos de los residuos frente a las variables regresoras y frente a los valores estimados junto con una curva de color azul indicando su tendencia; si vale FALSE simplemente da el valor del estadístico de Tukey y su $p$-valor.


Si optamos por ver los gráficos, entonces para concluir satisfactoriamente todos ellos deben parecer cielos estrellados.



## Linealidad.

Los gráficos de residuos parciales son una herramienta útil para detectar la no linealidad en una regresión. Se definen los residuos parciales $e_{ij}$ para la variable independiente $X_j$ como $$e_{ij}=e_i+b_jx_{ij},$$ 

donde $e_i$ es el residuo $i$-ésimo de la regresión lineal; $b_j$ es el coeficiente de $X_j$ en la regresión original y $x_{ij}$ es la observación $j$-ésima del individuo $i$-ésimo.

Los residuos parciales se dibujan contra los valores de $x_j$ y se hace su recta de regresión.

Si esta no se ajusta a la curva dada por una regresión no paramétrica suave (las variables independientes no están predeterminadas y se construyen con los datos), el modelo no es lineal.

### Linealidad en R

La función de $R$ para representar estos gráficos es ``crPlots`` del paquete **car**: ``crPlots(r)``.

## Observaciones anómalas.

Las observaciones anómalas pueden provocar que se malinterpreten patrones en el conjunto de datos. Además, puntos aislados pueden tener una gran influencia en el modelo de regresión, dando resultados completamente diferentes. Por ejemplo, pueden provocar que nuestro modelo no capture características importantes de los datos. 

Por ello, es importante detectarlas.

Existen tres tipos de observaciones anómalas:

* **Outliers de regresión** son observaciones que tienen un valor anómalo de la variable $Y$, condicionado a los valores de sus variables independientes $X_i$. Tendrán un residuo muy alto pero no pueden afectar demasiado a los coeficientes de la regresión.


* **Leverages.** son observaciones con un valor anómalo de las variables $X_i$. No tienen por qué afectar los coeficientes de la regresión.


* **Observaciones influyentes** son aquellas que tienen un leverage alto; son outliers de regresión y afectan fuertemente a la regresión.


### Leverages

Para hallar las observaciones que son leverages, necesitamos la matriz Hat: $H=X(X^TX)^{-1}X^T$. Como $\widehat{y}=Hy$, tenemos que $\widehat{y_j}=h_{1j}y_1+h_{2j}y_2+...+h_{nj}y_n=\sum_{i=1}^nh_{ij}x_i$ para $j=1,...,n$.

Si $h_{ij}$ es grande, la observación $i$-ésima tiene un impacto sustancial en el valor predicho $j$-ésimo.

Se define el leverage de la observación $i$ como su valor hat: $h_i=\sum_{j=1}^nh_{ij}^2$.
hat
La regla de decisión que nos dirá si una observación tiene leverage grande (y por lo tanto debe ser considerada con cuidado) es cuando su valor hat es mayor que $2(k+1)/n$.

La función ``hatvalues`` de R calcula los valores hat basándose en la regla anterior: ``which(hatvalues(lm1)>2(k+1)/n)``.

### Outliers

Para detectar outliers, en R se usa la ``función outlierTest`` del paquete **car**:  ``outlierTest(lm1)``. 

### Observaciones influyentes.

El método que usaremos es la llamada Distanca de Cook. Las observaciones influyentes son aquellas cuya distancia de Cook es mayor a $4/(n-k-1)$

En R, la distancia de Cook se calcula con ``cooks.distance`` del paquete **car**: ``which(cooks.distance(lm1)>4/(n-k-1))``

### Tratamiento de las observaciones anómalas.

El tratamiento suele ser complejo. 

Nos debemos preguntar si se deben a errores en la entrada o recogida de los datos y si este es el caso, eliminarlas.

Pero también pueden explicar que no se ha considerado alguna variable independiente que afecta al conjunto de observaciones anómalas.

Las más peligrosas son las influyentes.

En el supuesto de que se determine que se pueden eliminar, se tienen que eliminar de una a una actualizando el modelo cada vez. 